## Import Libraries

In [100]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC

from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import *

## Import Data

In [101]:
# Reading Processed Train Set Data File
raw_d1 = pd.read_csv('Data/Processed_Trainset.csv')

# Feature Headers List
headers = list(raw_d1.columns)

# Reading Test Data Set
raw_d2 = pd.read_csv('Data/Processed_Testset.csv')

id_col = list(raw_d2.Client_ID)
raw_d2.drop(['Client_ID'], axis=1, inplace=True)

## Correlation

In [102]:
# Generating Correlation Matrix
X_T = np.array(raw_d1)[:,:-1]
corr_mat = pd.DataFrame(np.corrcoef(X_T.T), columns=headers[:-1], index=headers[:-1])
corr_mat.to_csv('Data/Correlation_Matrix.csv')

## Formatting

In [103]:
# Convert Data into ndArrays
train_data = np.array(raw_d1)
test_data = np.array(raw_d2)

# Total Training Data
X_tot = train_data[:, :-1]
Y_tot = train_data[:, -1]

# Splitting Data into a Training Set and a Cross Validation Set
X_train, X_cross, Y_train, Y_cross = train_test_split(X_tot, Y_tot, test_size = 0.2, random_state = 42, shuffle = True)
X_train.shape, Y_train.shape

((19200, 31), (19200,))

## SMOTE

In [104]:
# categorical_features = list(range(0,18))

# smote = SMOTENC(categorical_features, sampling_strategy='auto', k_neighbors=10, n_jobs=1)
# X_train,Y_train =  smote.fit_resample(X_train, Y_train)

# pd.DataFrame(np.concatenate((X_train,np.reshape(Y_train, (Y_train.shape[0], 1))), axis=1)
#              ,columns=headers).to_csv('Data/SMOTED.csv', index=None)
# X_train.shape, Y_train.shape

## Normalizing Data

In [105]:
sc = RobustScaler()
sc.fit(X_tot)

X_tot = sc.fit_transform(X_tot)
X_train = sc.transform(X_train)
X_cross = sc.transform(X_cross)
test_data = sc.transform(test_data)

pd.DataFrame(X_tot, columns=headers[:-1])

,bl_ 500K,bl_1.5M,bl_100K,bl_1M,bl_2.5M,bl_200K,bl_300K,bl_400K,g_F,g_M,...,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-0.326428,-0.319355,-0.301720,-0.038562,1.072397,-0.478748
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,2.038682,0.726389,-0.145333,-0.004731,0.345448,0.763803
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,2.0,2.0,-2.0,-2.0,-0.235681,-0.229927,-0.230986,-0.362867,-0.371302,-0.353373
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.262710,0.310921,0.364248,0.437727,0.518774,0.567878
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,2.553482,2.612081,2.862625,3.218938,3.578009,3.716657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,2.600189,2.794818,3.247456,1.309661,0.268691,-0.022372
23996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,...,-1.0,-1.0,0.0,0.0,-0.322104,-0.313479,-0.286144,-0.192224,-0.264966,-0.353373
23997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,...,2.0,-1.0,0.0,0.0,-0.292738,-0.288615,-0.298980,0.033921,0.050387,0.047578
23998,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,-1.0,-0.374040,0.932376,0.969585,0.640112,-0.128412,0.660424


## Classification Model 

In [106]:
# rf = GradientBoostingClassifier(n_estimators= 300)

rf = RandomForestClassifier(bootstrap=True, class_weight= None, max_depth= 8, max_features= 30, max_leaf_nodes= None,
                            min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators= 100,
                            n_jobs= 1, oob_score= False, random_state= 85, verbose= 0, warm_start= False)
# rf = XGBClassifier()

rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features=30,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=85, verbose=0,
                       warm_start=False)

## Prediction and Evaluation

In [107]:
# Predicting Outcomes
predict_train = rf.predict(X_train)
predict_cross = rf.predict(X_cross)
predict_tot = rf.predict(X_tot)
predict_test = rf.predict(test_data)

# Computing Confusion Matrix on Cross Validation Set
confusion = confusion_matrix(Y_cross,predict_cross)

# Computing Accuracy Values
accuracy_train = accuracy_score(Y_train, predict_train)
accuracy_cross = accuracy_score(Y_cross, predict_cross)
accuracy_tot = accuracy_score(Y_tot, predict_tot)

# Computing AUC Values
auc_train = roc_auc_score(Y_train, predict_train)
auc_cross = roc_auc_score(Y_cross, predict_cross)
auc_tot = roc_auc_score(Y_tot, predict_tot)                

# Printing Accuracy Metrics
print("Train Accuracy            : {:.3f}%       AUC: {:.4f}".format(accuracy_train * 100, auc_train))
print("Cross Validation Accuracy : {:.3f}%       AUC: {:.4f} ".format(accuracy_cross * 100, auc_cross))
print("Total Set Accuracy        : {:.3f}%       AUC: {:.4f}".format(accuracy_tot * 100, auc_tot))
print("Confusion Matrix for Cross Validation Set :\n %s" % str(confusion))
print("Number of Ones Predicted for Test Data : %d" % np.sum(predict_test))

print(classification_report(Y_cross, predict_cross))

Train Accuracy            : 83.958%       AUC: 0.6810
Cross Validation Accuracy : 82.708%       AUC: 0.6614 
Total Set Accuracy        : 83.708%       AUC: 0.6771
Confusion Matrix for Cross Validation Set :
 [[3584  161]
 [ 669  386]]
Number of Ones Predicted for Test Data : 651
              precision    recall  f1-score   support

         0.0       0.84      0.96      0.90      3745
         1.0       0.71      0.37      0.48      1055

    accuracy                           0.83      4800
   macro avg       0.77      0.66      0.69      4800
weighted avg       0.81      0.83      0.81      4800



## Upload

In [82]:
# Uploading Predictions to a CSV
df = pd.DataFrame(predict_test, dtype=int)
df.index = id_col
df.to_csv('Data/Submission.csv', header=['NEXT_MONTH_DEFAULT'], index_label='Client_ID')